## 1、加载大模型参数

In [1]:
from unsloth import FastLanguageModel
import torch

# 准备一些配置
max_seq_length = 2048  # 最大序列长度
dtype = None           # 让Unsloth自动检测
load_in_4bit = True    # 4bit量化，节省显存

# 让我们选择Qwen2.5-Coder-7B-Instruct
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./qwen2.5-7B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 然后挂载LoRa
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,  # LoRa秩（秩越大，微调容量越大，同时资源消耗也更多）
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 09-11 09:46:50 [__init__.py:241] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.9: Fast Qwen2 patching. Transformers: 4.55.4. vLLM: 0.10.1.1.
   \\   /|    NVIDIA vGPU-32GB. Num GPUs = 1. Max memory: 31.484 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Unsloth 2025.8.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## 2、加载训练数据

In [2]:
import json
from datasets import load_dataset, load_from_disk, Dataset
from unsloth.chat_templates import get_chat_template, standardize_sharegpt
import re

system_prompt = "你是一个煤矿安全领域的专家，请你帮我回答一些有关煤矿安全领域的问题。"

with open('data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 将数据转换为Hugging Face Dataset格式
dataset = Dataset.from_dict({
    'conversations': data['conversations']
})

dataset = standardize_sharegpt(dataset)  # 把原本'from'/'value'结构转换成'role'/'content'

# 让tokenizer带上Qwen-2.5这种模板
tokenizer = get_chat_template(
    tokenizer,
    chat_template="qwen-2.5",
)


def replace_system_content_advanced(text, new_content):
    # 使用非贪婪匹配来精确匹配system部分
    pattern = r'(<\|im_start\|>system\n).*?(?=<\|im_end\|>)'
    return re.sub(pattern, r'\1' + new_content, text, flags=re.DOTALL)


def formatting_prompts_func(examples):
    """格式化提示，把conversations列表转换成text列表，符合Qwen-2.5的分隔符。"""
    convos = examples["conversations"]
    texts = [
        replace_system_content_advanced(
            tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False),
            system_prompt
        )
        for convo in convos
    ]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True,)
dataset[:2]

Unsloth: Standardizing formats (num_proc=128):   0%|          | 0/25520 [00:00<?, ? examples/s]

Map:   0%|          | 0/25520 [00:00<?, ? examples/s]

{'conversations': [[{'content': '除移动式的或辅助性的绞车外，提升装置的哪些部件需要满足最小直径与钢丝绳直径比值的要求？',
    'role': 'user'},
   {'content': '除移动式的或辅助性的绞车外，提升装置的天轮、滚筒、摩擦轮、导向轮和导向滚等部件需要满足最小直径与钢丝绳直径比值的要求。',
    'role': 'assistant'}],
  [{'content': '在煤矿安全法规中，关于预抽煤层瓦斯防突措施的区域，规定了哪项必须执行的效果验证程序？', 'role': 'user'},
   {'content': '必须对区域防突措施效果进行检验。', 'role': 'assistant'}]],
 'text': ['<|im_start|>system\n你是一个煤矿安全领域的专家，请你帮我回答一些有关煤矿安全领域的问题。<|im_end|>\n<|im_start|>user\n除移动式的或辅助性的绞车外，提升装置的哪些部件需要满足最小直径与钢丝绳直径比值的要求？<|im_end|>\n<|im_start|>assistant\n除移动式的或辅助性的绞车外，提升装置的天轮、滚筒、摩擦轮、导向轮和导向滚等部件需要满足最小直径与钢丝绳直径比值的要求。<|im_end|>\n',
  '<|im_start|>system\n你是一个煤矿安全领域的专家，请你帮我回答一些有关煤矿安全领域的问题。<|im_end|>\n<|im_start|>user\n在煤矿安全法规中，关于预抽煤层瓦斯防突措施的区域，规定了哪项必须执行的效果验证程序？<|im_end|>\n<|im_start|>assistant\n必须对区域防突措施效果进行检验。<|im_end|>\n']}

## 3、开始训练

In [3]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=4,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=4000,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        # output_dir="outputs",
        report_to="none",
    ),
)
trainer.train()

Unsloth: Tokenizing ["text"]:   0%|          | 0/25520 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 25,520 | Num Epochs = 11 | Total steps = 4,000
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 80,740,352 of 7,696,356,864 (1.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.433700
2,3.547100
3,3.377900
4,3.089100
5,2.713000
6,2.239300
7,1.954100
8,1.722900
9,1.545600
10,1.419400


TrainOutput(global_step=4000, training_loss=0.39620997847802936, metrics={'train_runtime': 19967.455, 'train_samples_per_second': 12.821, 'train_steps_per_second': 0.2, 'total_flos': 1.5558216224583352e+18, 'train_loss': 0.39620997847802936})

In [4]:
# from unsloth.chat_templates import train_on_responses_only

# trainer = train_on_responses_only(
#     trainer,
#     instruction_part="<|im_start|>user\n",
#     response_part="<|im_start|>assistant\n",
# )
# trainer.train()

In [5]:
# model.save_pretrained_merged("./lora_output/", tokenizer, "merged_16bit")
model.save_pretrained("./lora_output_2/")
tokenizer.save_pretrained("./lora_output_2")

('./lora_output_2/tokenizer_config.json',
 './lora_output_2/special_tokens_map.json',
 './lora_output_2/chat_template.jinja',
 './lora_output_2/vocab.json',
 './lora_output_2/merges.txt',
 './lora_output_2/added_tokens.json',
 './lora_output_2/tokenizer.json')

In [6]:
FastLanguageModel.for_inference(model)  # 启用推理优化

messages = [
    {"role": "user", "content": "爆破作业中，不同区域对照明电压的要求有何区别？"}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=64,
    temperature=1.5,
    use_cache=True
)

response = tokenizer.batch_decode(outputs)
print(response)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n爆破作业中，不同区域对照明电压的要求有何区别？<|im_end|>\n<|im_start|>assistant\n区别在于作业面积大的区域允许使用220V照明，而作业面积小的区域或硐室爆破必须使用36V以下电压。<|im_end|>']


In [7]:
# 方法2：使用partition方法
start_marker = '<|im_start|>assistant\n'
end_marker = '<|im_end|>'
response = response[0]
# 查找开始和结束位置
start_index = response.find(start_marker) + len(start_marker)
end_index = response.find(end_marker, start_index)

# 提取内容
content = response[start_index:end_index]
print(content)

区别在于作业面积大的区域允许使用220V照明，而作业面积小的区域或硐室爆破必须使用36V以下电压。
